# Welcome to the Prognostics Algorithms Package Tutorial

The goal of this notebook is to instruct the user on how to use and extend the NASA Python Prognostics Algorithms Package. 

First some background. The Prognostics Algorithms Package (`prog_algs`) contains tools for performing prognostics (event prediction) using the Prognostics Models Package. `prog_algs` also includes tools for analyzing the performance of prognostics algorithms. 

A few definitions:
* state estimation: The process of estimating the state of a system given sensor information
* prediction: The process of predicting the evolution of a systems state with time and the occurance of events. 

The `prog_algs` package has the following structure
* `prog_algs/state_estimators/` - Tools for performing state estimation
* `prog_algs/predictors/` - Tools for performing prediction
* `prog_algs/metrics/` - Tools for analyzing the performance of prognostics algorithms
* `prog_algs/samplers/` - Tools for sampling from a distribution

In addition to the `prog_algs` package, this repo includes examples showing how to use the package (see `examples/`), a template for implementing a new state estimator (`state_estimator_template`), a template for implementing a new predictor (`predictor_template`), documentation (`docs/`), and this tutorial (`tutorial.ipynb`).

Before you start, make sure that all the required packages are installed (defined in `requirements.txt`)

Now lets get started with some examples

## Prediction Example 
First thing to do is to import the prog_algs and the model you intend to use

In [1]:
import sys
sys.path.insert(1, "/Users/cteubert/Desktop/python-prognostics-models-package/")
from prog_models.models import battery_circuit
from prog_algs import *

Next, prepare the model like you did for simulation

In [2]:
def future_loading(t):
    # Variable (piece-wise) future loading scheme 
    if (t < 600):
        i = 2
    elif (t < 900):
        i = 1
    elif (t < 1800):
        i = 4
    elif (t < 3000):
        i = 2
    else:
        i = 3
    return {'i': i}

batt = battery_circuit.BatteryCircuit()

Now that we have our model ready, we can construct our state estimator:

In [3]:
filt = state_estimators.unscented_kalman_filter.UnscentedKalmanFilter(batt, batt.parameters['x0'])

The filter estimate function can then be run when there is updated data. Each iteration it will produce a new estimate of the system state (with uncertainty). For example:

In [5]:
print("Prior State:", filt.x.mean)
print('\tSOC: ', batt.event_state(filt.t, filt.x.mean)['EOD'])
example_measurements = {'t': 32.2, 'v': 3.915}
t = 0.1
filt.estimate(t, future_loading(t), example_measurements)
print("Posterior State:", filt.x.mean)
print('\tSOC: ', batt.event_state(filt.t, filt.x.mean)['EOD'])

Prior State: {'tb': 18.95, 'qb': 7856.3254, 'qcp': 0.0, 'qcs': 0.0}
	SOC:  1.0
Posterior State: {'tb': 31.016216083371525, 'qb': 7856.035209074807, 'qcp': 0.3823629075923045, 'qcs': 0.15933015979554213}
	SOC:  0.9999626860067903


That's the state estimation step- now lets prepare for prediction. 

In [8]:
mc = predictors.monte_carlo.MonteCarlo(batt)
samples = filt.x.sample(20)
prediction_config = {'dt': 0.025}

Now lets use the constructed mc predictor to perform a single prediction. Note this may take up to a minute

In [9]:
(times, inputs, states, outputs, event_states, eol) = mc.predict(samples, future_loading, prediction_config)


Next, let's use the metrics package to analyse the results

In [13]:
print("\nEOD Predictions (s):")
from prog_algs.metrics import samples as metrics 
print('\tPercentage between 3005.2 and 3005.6: ', metrics.percentage_in_bounds(eol, [3005.2, 3005.6])*100.0, '%')
print('\tAssuming ground truth 3005.25: ', metrics.eol_metrics(eol, 3005.25))
print('\tP(Success) if mission ends at 3005.25: ', metrics.prob_success(eol, 3005.25))


EOD Predictions (s):
	Percentage between 3005.2 and 3005.6:  90.0 %
	Assuming ground truth 3005.25:  {'min': 3005.1250000065497, 'percentiles': {'0.01': None, '0.1': None, '1': None, '10': 3005.20000000655, '25': 3005.25000000655, '50': 3005.3250000065505, '75': 3005.450000006551}, 'median': 3005.3250000065505, 'mean': 3005.3375000065503, 'std': 0.12285662375350309, 'max': 3005.5750000065514, 'median absolute deviation': 0.10250000000037289, 'mean absolute deviation': 0.10500000000033652, 'number of samples': 20, 'mean absolute error': 0.12000000327552698, 'mean absolute percentage error': 3.993012337593444e-05, 'relative accuracy': 0.9999708842836535, 'ground truth percentile': 25.0}
	P(Success) if mission ends at 3005.25:  0.75


## Conclusion
Thank you for trying out this tutorial. See the examples in the `examples/` folder for more details on how to use the package. Any questions can be directed to Chris Teubert (christopher.a.teubert@nasa.gov)